## Course: DSC550
## Assignment: 11.2 Project Milestone 5
## Name: Laura Hoffmann
## Date: 5/30/2021

#### Import the Libraries

In [1]:
import pandas as pd

from sklearn.preprocessing import OrdinalEncoder

import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np

# Milestone 3/4
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from yellowbrick.classifier import ConfusionMatrix
from yellowbrick.classifier import ClassificationReport
from sklearn import tree
from sklearn import metrics
from collections import Counter
from sklearn.preprocessing import LabelEncoder

# Milestone 1

In [2]:
# Read in the data
data = pd.read_csv('strokedata.csv')

In [3]:
# Rename columns to get a cleaner feel on the data set
data = data.rename(columns = {'id': 'ID', 'gender': 'Gender', 'age': 'Age', 'hypertension': 'Hypertension',
                             'heart_disease': 'Heart_Disease', 'ever_married': 'Married', 'work_type': 'Work',
                             'Residence_type': 'Residence', 'avg_glucose_level': 'Glucose', 'bmi': 'BMI',
                             'smoking_status': 'Smoking_Status', 'stroke': 'Stroke'}, inplace = False)

In [4]:
# Print the dimensions of the df
print('Dimensions:',data.shape)
# Print the first few rows of the df
data.head()

Dimensions: (5110, 13)


,Unnamed: 0,ID,Gender,Age,Hypertension,Heart_Disease,Married,Work,Residence,Glucose,BMI,Smoking_Status,Stroke
0,0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


In [5]:
# Find what types of variables are in the df
# Numerical data
print("Describe Data:")
print(round(data.describe(),2))
print("*"*70)
# Text data
print("Summarized Data:")
print(data.describe(include=['O']))

Describe Data:
       Unnamed: 0        ID      Age  Hypertension  Heart_Disease  Glucose  \
count     5110.00   5110.00  5110.00        5110.0        5110.00  5110.00   
mean      2554.50  36517.83    43.23           0.1           0.05   106.15   
std       1475.27  21161.72    22.61           0.3           0.23    45.28   
min          0.00     67.00     0.08           0.0           0.00    55.12   
25%       1277.25  17741.25    25.00           0.0           0.00    77.24   
50%       2554.50  36932.00    45.00           0.0           0.00    91.88   
75%       3831.75  54682.00    61.00           0.0           0.00   114.09   
max       5109.00  72940.00    82.00           1.0           1.00   271.74   

           BMI   Stroke  
count  4909.00  5110.00  
mean     28.89     0.05  
std       7.85     0.22  
min      10.30     0.00  
25%      23.50     0.00  
50%      28.10     0.00  
75%      33.10     0.00  
max      97.60     1.00  
************************************************

In [ ]:
# Set up the figure size
plt.rcParams['figure.figsize'] = (20, 5)

# Make subplots
fig, axes = plt.subplots(nrows = 1, ncols = 2)

# X axes, colors and bins
x_axes = ['Age', 'Glucose'] # Histograms for the columns Age and Glucose
colors = ['powderblue', 'tomato']
bins = [40, 10] # Respective bin sizes

# Make the graphs
axes = axes.ravel()
for idx, ax in enumerate(axes):
    ax.hist(data[x_axes[idx]].dropna(), bins=bins[idx], color=colors[idx])
    ax.set_xlabel(x_axes[idx], fontsize=20)
    ax.set_ylabel('Counts', fontsize=20)
    ax.tick_params(axis='both', labelsize=15)
plt.show()

In [ ]:
# Histogram for BMI
plt.rcParams['figure.figsize'] = (10, 5)

plt.hist(data['BMI'].dropna(), bins=25, color='plum')
plt.xlabel('BMI', fontsize=20)
plt.ylabel('Counts', fontsize=20)
plt.tick_params(axis='both', labelsize=15)
plt.show()

In [ ]:
strokebmi = data.loc[data['Stroke'].dropna() == 1, 'BMI']
nonstrokebmi = data.loc[data['Stroke'].dropna() == 0, 'BMI']

strokeglucose = data.loc[data['Stroke'].dropna() == 1, 'Glucose']
nonstrokeglucose = data.loc[data['Stroke'].dropna() == 0, 'Glucose']

In [ ]:
# Set plot size
plt.rcParams['figure.figsize'] = (10, 8)
fig = plt.figure()
ax = fig.add_subplot()

# Add data to the scatter plot for strokes and nonstrokes
plt.scatter(strokebmi, strokeglucose, color='tomato', label='Stroke', s=4)
plt.scatter(nonstrokebmi, nonstrokeglucose, color='powderblue', label='No Stroke', s=4)

# Labels
plt.title('Stroke Scatterplot for BMI vs. Glucose', fontsize=25)
plt.xlabel('BMI', fontsize=20)
plt.ylabel('Glucose', fontsize=20)
plt.legend(loc=1, fontsize=20)
plt.show()

In [ ]:
# Set up the figure size
plt.rcParams['figure.figsize'] = (20, 30)

# Make subplots
fig, axes = plt.subplots(nrows = 4, ncols = 2)

# GENDER: Make the data read to feed into the visulizer
Stroke1 = data.replace({'Stroke': {1: 'Stroke', 0: 'No Stroke'}})[data['Stroke']==1]['Gender'].value_counts()
Stroke0 = data.replace({'Stroke': {1: 'Stroke', 0: 'No Stroke'}})[data['Stroke']==0]['Gender'].value_counts()
Stroke0 = Stroke0.reindex(index=Stroke1.index)

# GENDER: Make the bar plot
p1 = axes[0, 0].bar(Stroke1.index, Stroke1.values, color='tomato')
p2 = axes[0, 0].bar(Stroke0.index, Stroke0.values, color='powderblue', bottom=Stroke1.values)
axes[0, 0].set_title('Gender', fontsize=25)
axes[0, 0].set_ylabel('Counts', fontsize=20)
axes[0, 0].tick_params(axis='both', labelsize=15)
axes[0, 0].legend((p1[0], p2[0]), ('Stroke', 'No Stroke'), fontsize = 15)


# MARRIED: Make the data read to feed into the visulizer
Married1 = data.replace({'Stroke': {1: 'Stroke', 0: 'No Stroke'}})[data['Stroke']==1]['Married'].value_counts()
Married0 = data.replace({'Stroke': {1: 'Stroke', 0: 'No Stroke'}})[data['Stroke']==0]['Married'].value_counts()
Married0 = Married0.reindex(index = Married1.index)

# MARRIED: Make the bar plot
p3 = axes[0, 1].bar(Married1.index, Married1.values, color='tomato')
p4 = axes[0, 1].bar(Married0.index, Married0.values, color='powderblue', bottom=Married1.values)
axes[0, 1].set_title('Married', fontsize=25)
axes[0, 1].set_ylabel('Counts', fontsize=20)
axes[0, 1].tick_params(axis='both', labelsize=15)
axes[0, 1].legend((p3[0], p4[0]), ('Stroke', 'No Stroke'), fontsize = 15)


# WORK: Make the data read to feed into the visulizer
Work1 = data.replace({'Stroke': {1: 'Stroke', 0: 'No Stroke'}})[data['Stroke']==1]['Work'].value_counts()
Work0 = data.replace({'Stroke': {1: 'Stroke', 0: 'No Stroke'}})[data['Stroke']==0]['Work'].value_counts()
Work0 = Work0.reindex(index = Work1.index)

# WORK: Make the bar plot
p5 = axes[1, 0].bar(Work1.index, Work1.values, color='tomato')
p6 = axes[1, 0].bar(Work0.index, Work0.values, color='powderblue', bottom=Work1.values)
axes[1, 0].set_title('Work', fontsize=25)
axes[1, 0].set_ylabel('Counts', fontsize=20)
axes[1, 0].tick_params(axis='both', labelsize=15)
axes[1, 0].legend((p5[0], p6[0]), ('Stroke', 'No Stroke'), fontsize = 15)


# RESIDENCE: Make the data read to feed into the visulizer
Residence1 = data.replace({'Stroke': {1: 'Stroke', 0: 'No Stroke'}})[data['Stroke']==1]['Residence'].value_counts()
Residence0 = data.replace({'Stroke': {1: 'Stroke', 0: 'No Stroke'}})[data['Stroke']==0]['Residence'].value_counts()
Residence0 = Residence0.reindex(index = Residence1.index)

# RESIDENCE: Make the bar plot
p7 = axes[1, 1].bar(Residence1.index, Residence1.values, color='tomato')
p8 = axes[1, 1].bar(Residence0.index, Residence0.values, color='powderblue', bottom=Residence1.values)
axes[1, 1].set_title('Residence', fontsize=25)
axes[1, 1].set_ylabel('Counts', fontsize=20)
axes[1, 1].tick_params(axis='both', labelsize=15)
axes[1, 1].legend((p7[0], p8[0]), ('Stroke', 'No Stroke'), fontsize = 15)


# SMOKER: Make the data read to feed into the visulizer
Smoke1 = data.replace({'Stroke': {1: 'Stroke', 0: 'No Stroke'}})[data['Stroke']==1]['Smoking_Status'].value_counts()
Smoke0 = data.replace({'Stroke': {1: 'Stroke', 0: 'No Stroke'}})[data['Stroke']==0]['Smoking_Status'].value_counts()
Smoke0 = Smoke0.reindex(index = Smoke1.index)

# SMOKER: Make the bar plot
p9 = axes[2, 0].bar(Smoke1.index, Smoke1.values, color='tomato')
p10 = axes[2, 0].bar(Smoke0.index, Smoke0.values, color='powderblue', bottom=Smoke1.values)
axes[2, 0].set_title('Smoking Status', fontsize=25)
axes[2, 0].set_ylabel('Counts', fontsize=20)
axes[2, 0].tick_params(axis='both', labelsize=15)
axes[2, 0].legend((p9[0], p10[0]), ('Stroke', 'No Stroke'), fontsize = 15)


# HYPERTENSION: Make the data read to feed into the visulizer
# Have to replace hypertension numbers (0=no hypertension, 1=hypertension)
Hypertension1 = data.replace({'Stroke': {1: 'Stroke', 0: 'No Stroke'}}).replace({'Hypertension': {0: 'No Hypertension', 1: 'Hypertension'}})[data['Stroke']==1]['Hypertension'].value_counts()
Hypertension0 = data.replace({'Stroke': {1: 'Stroke', 0: 'No Stroke'}}).replace({'Hypertension': {0: 'No Hypertension', 1: 'Hypertension'}})[data['Stroke']==0]['Hypertension'].value_counts()
Hypertension0 = Hypertension0.reindex(index = Hypertension1.index)

# HYPERTENSION: Make the bar plot
p11 = axes[2, 1].bar(Hypertension1.index, Hypertension1.values, color='tomato')
p12 = axes[2, 1].bar(Hypertension0.index, Hypertension0.values, color='powderblue', bottom=Hypertension1.values)
axes[2, 1].set_title('Hypertension', fontsize=25)
axes[2, 1].set_ylabel('Counts', fontsize=20)
axes[2, 1].tick_params(axis='both', labelsize=15)
axes[2, 1].legend((p11[0], p12[0]), ('Stroke', 'No Stroke'), fontsize = 15)


# HEART DISEASE: Make the data read to feed into the visulizer
# Have to replace hypertension numbers (0=no heart disease, 1=heart disease)
HD1 = data.replace({'Stroke': {1: 'Stroke', 0: 'No Stroke'}}).replace({'Heart_Disease': {0: 'No Heart Disease', 1: 'Heart Disease'}})[data['Stroke']==1]['Heart_Disease'].value_counts()
HD0 = data.replace({'Stroke': {1: 'Stroke', 0: 'No Stroke'}}).replace({'Heart_Disease': {0: 'No Heart Disease', 1: 'Heart Disease'}})[data['Stroke']==0]['Heart_Disease'].value_counts()
HD0 = HD0.reindex(index = HD1.index)

# HEART DISEASE: Make the bar plot
p11 = axes[3, 0].bar(HD1.index, HD1.values, color='tomato')
p12 = axes[3, 0].bar(HD0.index, HD0.values, color='powderblue', bottom=HD1.values)
axes[3, 0].set_title('Heart Disease', fontsize=25)
axes[3, 0].set_ylabel('Counts', fontsize=20)
axes[3, 0].tick_params(axis='both', labelsize=15)
axes[3, 0].legend((p11[0], p12[0]), ('Stroke', 'No Stroke'), fontsize = 15)

I liked these graphs a lot better than the originals because they seem easier to interpret. Based on these graphs it looks like males and females are almost equally likely to have strokes, where males may be slightly more likely. Married people seem to have more strokes than non married people because the bar for total married people is about doubled than the non married but the number of strokes in married people seems about 5x the number of strokes in non married people. Working as self employed or for the government might slightly increase the risk, residence area seems to not affect the risk very much, but obviously smoking is a big factor.

Each of these graphs reveals just how much each feature can affect the risk of stroke. Obviously there are some things the patients can do to work on a lower stroke risk, by living a more healthy lifestyle they could lower their BMI, and quit smoking to lower their risk. However there are other factors, such as being married that would have an affect on the risk of stroke that would be more difficult to change. I believe in that case in particular, it would be more about moving toward a healthy relationship that lowers stress. Overall, most of these are still factors that could affect the risk of stroke and by using them and a model to predict the likelyhood of a stroke. Then other factors could be adjusted to prevent that happening.

# Milestone 2

In [ ]:
data1= data
# Dropping ID and Unnamed column because they contain no relevant data
data1 = data1.drop(['Unnamed: 0', 'ID'], axis=1)

ord_enc = OrdinalEncoder()

# Encode categorical data into numerical for feature analysis and reduction
data1['Gender'] = ord_enc.fit_transform(data1[['Gender']])
data1['Married'] = ord_enc.fit_transform(data1[['Married']])
data1['Work'] = ord_enc.fit_transform(data1[['Work']])
data1['Residence'] = ord_enc.fit_transform(data1[['Residence']])
data1['Smoking_Status'] = ord_enc.fit_transform(data1[['Smoking_Status']])

data1.head(3)

### Checking and adjusting missing data

In [ ]:
# Check for missing values
data1.isnull().sum()

In [ ]:
# Replace missing values with mean for BMI
# Define function
def na_mean(data, inplace=True):
    return data.fillna(round(data.mean(),2), inplace=inplace)

# Fill BMI nan values
na_mean(data1['BMI'])

data1['BMI'].describe()

In [ ]:
# Recheck
data1['BMI'].isnull().sum()

### Feature reduction check

In [ ]:
# Using Pearson Correlation
plt.figure(figsize=(12,10))
cor = data1.corr()
sns.heatmap(round(cor,1), annot=True, cmap=plt.cm.Blues)
plt.show()

None of the varaibles are so closely correlated with each other that I'd want to remove them now, for being almost identicle only one would suffice but none are too close. Marriage and age are the two most closely correlated with each other but I'm choosing to keep both because of how they impact the outcome of stroke differently. Obviously age is a major factor when it comes to the possibility of stroke but the stress of marriage or lack thereof can also play a role. 

In [ ]:
# Correlation with output variable
cor = data1.corr()
cor_target = abs(cor["Stroke"])

# Selecting highly correlated features
relevant_features = cor_target[cor_target>0.04]
relevant_features

Above I tested correlations with the target variable (Stroke) to see which features or columns might be irrelevant if they had close correlations with each other. Then I reviewed the correlations between the features and the target varaible to see which are most highly correlated with Stroke. Gender and residence were the lowest correlated features with the target variable, Stroke.

In [ ]:
# Dropping columns with no relevant data in original dataset
data = data.drop(['Unnamed: 0','ID'], axis=1)
# Replacing the missing values in data BMI with the mean
na_mean(data['BMI'])

### Adjust skewed data

In [ ]:
# Set up the figure size
plt.rcParams['figure.figsize'] = (20, 5)

# Make subplots
fig, axes = plt.subplots(nrows = 1, ncols = 2)

# X axes, colors and bins
x_axes = ['Glucose', 'BMI'] # Histograms for the columns Age and Glucose
colors = ['powderblue', 'tomato']
bins = [40, 30] # Respective bin sizes

# Make the graphs
axes = axes.ravel()
for idx, ax in enumerate(axes):
    ax.hist(data[x_axes[idx]].dropna(), bins=bins[idx], color=colors[idx])
    ax.set_xlabel(x_axes[idx], fontsize=20)
    ax.set_ylabel('Counts', fontsize=20)
    ax.tick_params(axis='both', labelsize=15)
plt.show()

Here we have two variables that are right skewed so I'll adjust them using the logarithms of both of these features for transformation.

In [ ]:
def log_transformation(data):
    return round(data.apply(np.log),2)

In [ ]:
data['Glucose_log'] = log_transformation(data['Glucose'])
data['BMI_log'] = log_transformation(data['BMI'])

In [ ]:
# Glucose log histogram
plt.rcParams['figure.figsize'] = (10, 5)

plt.hist(data['Glucose_log'], bins=40, color='powderblue')
plt.title('Glucose Log Distribution', fontsize=25)
plt.xlabel('Glucose_log', fontsize=20)
plt.ylabel('Counts', fontsize=20)
plt.tick_params(axis='both', labelsize=15)
plt.show()

In [ ]:
# BMI log histogram
plt.rcParams['figure.figsize'] = (10, 5)

plt.hist(data['BMI_log'], bins=40, color='tomato')
plt.title('BMI Log Distribution', fontsize=25)
plt.xlabel('BMI_log', fontsize=20)
plt.ylabel('Counts', fontsize=20)
plt.tick_params(axis='both', labelsize=15)
plt.show()

In [ ]:
data.head()

### Create dummy variables for categorical data

In [ ]:
# categorical features list
data['Married'] = ord_enc.fit_transform(data[['Married']])
cat_features = ['Gender', 'Hypertension', 'Heart_Disease', 'Married', 'Work', 'Smoking_Status']
data_cat = data[cat_features]

# Not necessary to make dummy variables for this as it is already encoded properly
#data_cat = data_cat.replace({'Hypertension': {0: 'No', 1: 'Yes'}})
#data_cat = data_cat.replace({'Heart_Disease': {0: 'No', 1: 'Yes'}})

In [ ]:
data_cat_dummies = pd.get_dummies(data_cat)

In [ ]:
data_cat_dummies.head()

# Milestone 3

Below was progress made before dealing with the imbalanced data in the data set. So the work from this milestone was commented out and then later applied to the data after dealing with the imbalanced issue.

### Logistic Regression

In [ ]:
# # create a whole features dataset that can be used for train and validation data splitting
# features_model = ['Age', 'Glucose_log', 'BMI_log']
# data_model_X = pd.concat([data[features_model], data_cat_dummies], axis=1)
# data_model_X.head()

In [ ]:
# # create a whole target dataset that can be used for train and validation data splitting
# data_model_y = data.replace({'Stroke': {1: 'Stroke', 0: 'No_Stroke'}})['Stroke']

In [ ]:
# # split the data
# X_train, X_test, y_train, y_test = train_test_split(data_model_X, data_model_y, test_size =0.30, random_state=11)

In [ ]:
# # number of samples in each set
# print("Number of samples in training set: ", X_train.shape[0])
# print("Number of samples in validation set:", X_test.shape[0])

In [ ]:
# # Stroke and no stroke (training set)
# print('Number of strokes and non strokes in the training set:')
# print(y_train.value_counts())

In [ ]:
# # Survived and not-survived (testing set)
# print('Number of strokes and non strokes in the testing set:')
# print(y_test.value_counts())

In [ ]:
# # Instantiate the classification model 
# model = LogisticRegression(solver='liblinear')

# #The ConfusionMatrix visualizer taxes a model
# classes = ['No Stroke','Stroke']
# cm = ConfusionMatrix(model, classes=classes)

# #Fit fits the passed model. This is unnecessary if you pass the visualizer a pre-fitted model
# cm.fit(X_train, y_train)

# #To create the ConfusionMatrix, we need some test data. Score runs predict() on the data
# #and then creates the confusion_matrix from scikit learn.
# cm.score(X_test, y_test)

# # change fontsize of the labels in the figure
# for label in cm.ax.texts:
#     label.set_size(20)

# #How did we do?
# cm.poof()

In [ ]:
# # set the size of the figure and the font size 
# %matplotlib inline
# plt.rcParams['figure.figsize'] = (15, 7)
# plt.rcParams['font.size'] = 20

# # Instantiate the visualizer
# visualizer = ClassificationReport(model, classes=classes)

# visualizer.fit(X_train, y_train)  # Fit the training data to the visualizer
# visualizer.score(X_test, y_test)  # Evaluate the model on the test data
# g = visualizer.poof()             # Draw/show/poof the data

### Short conclusion
Obviously logistic regression is not a great machine learning algorithm for the data set, it missed predicting literally any strokes. I'm believing it's primarily because of the lower correlation coefficients between the predictor variables and the target variable. Even though this algorithm is 95% accurate it's still lacking because of all of the strokes it failed to predict.

### Decision Tree Classifier

In [ ]:
# object_type_columns = data.select_dtypes(include='object')
# le = LabelEncoder()
# for title in object_type_columns:
#     data[title] = le.fit_transform(data[title])

In [ ]:
# X = data.drop(['Stroke'], axis=1)
# y = data['Stroke']

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
# print('absolut values for train set: ', Counter(y_train), sep='\n')
# print()
# print('in percents for train set: ', y_train.value_counts(normalize=True).round(2) * 100, sep='\n')

In [ ]:
# DT_clf = tree.DecisionTreeClassifier()
# DT_clf.fit(X_train, y_train)

In [ ]:
# DT_prediction = DT_clf.predict(X_test)
# DT_metrics = metrics.f1_score(y_test, DT_prediction).round(2)
# DT_report = metrics.classification_report(y_test, DT_prediction)

In [ ]:
# #The ConfusionMatrix visualizer taxes a model
# classes = ['No Stroke','Stroke']
# cm = ConfusionMatrix(DT_clf, classes=classes)

# #Fit fits the passed model. This is unnecessary if you pass the visualizer a pre-fitted model
# cm.fit(X_train, y_train)

# #To create the ConfusionMatrix, we need some test data. Score runs predict() on the data
# #and then creates the confusion_matrix from scikit learn.
# cm.score(X_test, y_test)

# # change fontsize of the labels in the figure
# for label in cm.ax.texts:
#     label.set_size(20)

# #How did we do?
# cm.poof()

In [ ]:
# print(DT_report)

## Short conclusion

While logistic regression may have a higher accuracy rate I actually like the results from the decision tree more because it caught more of the strokes than logistic regression. Even though there are more false positives I believe in this case it would be better to stick with the lower accuracy but be able to predict more of the strokes. I will try to tune this machine learning algorithm in turn to possibly increase the accuracy. After all at least this ML algorithm predicted strokes at all. 

# Milestone 4

## Dealing with Imbalanced Data

In [ ]:
# find number of positives and negatives
y = data.Stroke

df_majority = data[data.Stroke==0]
df_minority = data[data.Stroke==1]

nclass0 = len(df_majority)
print("Number of cases with no stroke:", nclass0)
nclass1 = len(df_minority)
print("Number of cases with a stroke:", nclass1)

As we can see the classes are very imbalanced so I will correct this by downsampling the cases with no stroke. 

In [ ]:
# pip install imblearn

In [ ]:
# from imblearn.over_sampling import SMOTE

# # Resample the minority class. You can change the strategy to 'auto' if you are not sure.
# sm = SMOTE(sampling_strategy='minority', random_state=7)

# # Fit the model to generate the data.
# oversampled_trainX, oversampled_trainY = sm.fit_resample(data.drop('Stroke', axis=1), data['Stroke'])
# oversampled_train = pd.concat([pd.DataFrame(oversampled_trainY), pd.DataFrame(oversampled_trainX)], axis=1)
# oversampled_train.columns = normalized_df.columns

Downsampling

In [ ]:
# downsample the number of negative cases
from sklearn.utils import resample
df_majority_downsampled = resample(df_majority, 
                                 replace=False,    # sample without replacement
                                 n_samples=249)     # to match minority class

In [ ]:
# new data frame with less data points
ds_df = pd.concat([df_majority_downsampled, df_minority])
ds_df

## Logistic Regression

In [ ]:
dsdata_cat = ds_df[cat_features]
data_cat_dummies = pd.get_dummies(dsdata_cat)

In [ ]:
# create a whole features dataset that can be used for train and validation data splitting
features_model = ['Age', 'Glucose_log', 'BMI_log']
data_model_X = pd.concat([ds_df[features_model], data_cat_dummies], axis=1)
data_model_X.head()

In [ ]:
# create a whole target dataset that can be used for train and validation data splitting
data_model_y = ds_df.replace({'Stroke': {1: 'Stroke', 0: 'No_Stroke'}})['Stroke']

In [ ]:
# split the data
X_train, X_test, y_train, y_test = train_test_split(data_model_X, data_model_y, test_size =0.25, random_state=27)

In [ ]:
# number of samples in each set
print("Number of samples in training set: ", X_train.shape[0])
print("Number of samples in validation set:", X_test.shape[0])

In [ ]:
# Stroke and no stroke (training set)
print('Number of strokes and non strokes in the training set:')
print(y_train.value_counts())

In [ ]:
# Survived and not-survived (testing set)
print('Number of strokes and non strokes in the testing set:')
print(y_test.value_counts())

In [ ]:
# Instantiate the classification model 
model = LogisticRegression(solver='liblinear')

#The ConfusionMatrix visualizer taxes a model
classes = ['No Stroke','Stroke']
cm = ConfusionMatrix(model, classes=classes)

#Fit fits the passed model. This is unnecessary if you pass the visualizer a pre-fitted model
cm.fit(X_train, y_train)

#To create the ConfusionMatrix, we need some test data. Score runs predict() on the data
#and then creates the confusion_matrix from scikit learn.
cm.score(X_test, y_test)

# change fontsize of the labels in the figure
for label in cm.ax.texts:
    label.set_size(20)

#How did we do?
cm.poof()

In [ ]:
# set the size of the figure and the font size 
%matplotlib inline
plt.rcParams['figure.figsize'] = (15, 7)
plt.rcParams['font.size'] = 20

# Instantiate the visualizer
visualizer = ClassificationReport(model, classes=classes)

visualizer.fit(X_train, y_train)  # Fit the training data to the visualizer
visualizer.score(X_test, y_test)  # Evaluate the model on the test data
g = visualizer.poof()             # Draw/show/poof the data

## Decision Tree Classifier

In [ ]:
object_type_columns = ds_df.select_dtypes(include='object')
le = LabelEncoder()
for title in object_type_columns:
    ds_df[title] = le.fit_transform(ds_df[title])

In [ ]:
# separate the target variable
X = ds_df.drop(['Stroke'], axis=1)
y = ds_df['Stroke']

In [ ]:
# splitting into training and testing groups
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25, random_state=27)
print('absolut values for train set: ', Counter(y_train), sep='\n')
print()
print('in percents for train set: ', y_train.value_counts(normalize=True).round(2) * 100, sep='\n')

In [ ]:
DT_clf = tree.DecisionTreeClassifier()
DT_clf.fit(X_train, y_train)

In [ ]:
DT_prediction = DT_clf.predict(X_test)
DT_metrics = metrics.f1_score(y_test, DT_prediction).round(2)
DT_report = metrics.classification_report(y_test, DT_prediction)

In [ ]:
#The ConfusionMatrix visualizer taxes a model
classes = ['No Stroke','Stroke']
cm = ConfusionMatrix(DT_clf, classes=classes)

#Fit fits the passed model. This is unnecessary if you pass the visualizer a pre-fitted model
cm.fit(X_train, y_train)

#To create the ConfusionMatrix, we need some test data. Score runs predict() on the data
#and then creates the confusion_matrix from scikit learn.
cm.score(X_test, y_test)

# change fontsize of the labels in the figure
for label in cm.ax.texts:
    label.set_size(20)

#How did we do?
cm.poof()

In [ ]:
print(DT_report)

Now the the imbalanced issue is dealt with, working toward tuning the machine learning algorithm will be slightly easier. The logistic regression algorithm is no longer predicting 0 strokes which is a good sign. For the next milestone I will try tuning the algorithms to increase precision of the target variable.

# Milestone 5

## Random Forrest

In [ ]:
from sklearn import preprocessing 
from sklearn.linear_model import BayesianRidge
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# create a whole features dataset that can be used for train and validation data splitting
features_model = ['Age', 'Glucose_log', 'BMI_log']
data_model_X = pd.concat([data[features_model], data_cat_dummies], axis=1)
data_model_X.head()

In [ ]:
# create a whole target dataset that can be used for train and validation data splitting
data_model_y = data.replace({'Stroke': {1: 'Stroke', 0: 'No_Stroke'}})['Stroke']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=4)
print ('Train set:', X_train.shape,  y_train.shape)
print ('Test set:', X_test.shape,  y_test.shape)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import itertools
from sklearn.metrics import f1_score
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)
    plt.figure(figsize=(5,4))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
RF = RandomForestClassifier( max_depth= 10, random_state=0, n_estimators=10)
RFF = RF.fit(X_train, y_train)
#Get the Score of Random Forest Classifier both Train and Test
RFM_Train = RF.score(X_train, y_train)
RFM_Test = RF.score(X_test, y_test)
print('Random Forest Train Score: ' + str(RFM_Train))
print('Random Forest Test Score: ' + str(RFM_Test))
#Predict value of RFM
RFM_pred = RF.predict(X_test)
print(RFM_pred[0:5])

In [ ]:
from sklearn.metrics import f1_score
f1_score(y_test, RFM_pred, average='weighted') 
# Compute confusion matrix  
RFM_matrix = confusion_matrix(y_test, RFM_pred)
np.set_printoptions(precision=2)

print (classification_report(y_test, RFM_pred))

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(RFM_matrix, classes=['No Stroke','Stroke'],normalize= False,  title='Random Forest Confusion matrix')